In [1]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option('max.columns', 100)

%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [151]:
df = pd.read_excel('sem_duplicate.xlsx')

In [152]:
df.loc[:130]

,title,image,classification,url,reviews,target
0,"Artificial Intelligence, Machine Learning, and...",https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Artificial-Intelligence...,1,1
1,Model-Based Clustering and Classification for ...,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Model-Based-Clustering-...,1,1
2,Spatial Modeling in GIS and R for Earth and En...,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Spatial-Modeling-Earth-...,1,1
3,Statistical Machine Learning: A Unified Framew...,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Statistical-Machine-Lea...,1,1
4,Data Science for Transport: A Self-Study Guide...,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Data-Science-Transport-...,1,0
...,...,...,...,...,...,...
126,Data Science for Business: What You Need to Kn...,https://images-na.ssl-images-amazon.com/images...,4.5,https://www.amazon.com/Data-Science-Business-D...,505,0
127,Python for Data Analysis: Data Wrangling with ...,https://images-na.ssl-images-amazon.com/images...,4.5,https://www.amazon.com/Python-Data-Analysis-Wr...,579,0
128,Deep Learning with Python,https://images-na.ssl-images-amazon.com/images...,4.5,https://www.amazon.com/Deep-Learning-Python-Fr...,601,1
129,The Self-Taught Programmer: The Definitive Gui...,https://images-na.ssl-images-amazon.com/images...,4.5,https://www.amazon.com/Self-Taught-Programmer-...,616,0


In [153]:
colunas = ['title','classification','reviews']
limpo = df[colunas]

In [106]:
def f(row):
    if row['classification'] >= 3.5 and row['reviews'] >= 50:
        val = 1
    else:
        val = 0
    return val

limpo['target'] = limpo.apply(f, axis=1)

C:\Users\Gustavo\Anaconda3\envs\to_deploy\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [107]:
limpo['target'].value_counts()

0    204
1     68
Name: target, dtype: int64

In [108]:
limpo

,title,classification,reviews,target
0,97 Things About Ethics Everyone in Data Scienc...,5.0,3,0
1,A Common-Sense Guide to Data Structures and Al...,4.5,8,0
2,A Field Guide to Lies: Critical Thinking with ...,4.3,22,0
3,A First Course in Machine Learning (Machine Le...,4.1,21,0
4,A PRACTITIONER'S GUIDE TO BUSINESS ANALYTICS: ...,3.9,59,1
...,...,...,...,...
267,"Truthful Art, The: Data, Charts, and Maps for ...",4.8,121,1
268,Ultimate Step by Step Guide to Machine Learnin...,4.3,79,1
269,Understanding Advanced Statistical Methods (Ch...,4.4,12,0
270,Understanding Machine Learning: From Theory to...,4.1,94,1


In [154]:
y = df['target']
#y = limpo['target']

In [155]:
mask = ['classification','reviews']
treinar = limpo[mask]
treinar

,classification,reviews
0,5.0,1
1,5.0,1
2,5.0,1
3,5.0,1
4,5.0,1
...,...,...
267,3.1,12
268,3.0,1
269,3.0,2
270,2.9,5


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [42]:
from sklearn.model_selection import train_test_split

In [156]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['title']

In [157]:
df_limpo

,title
0,"Artificial Intelligence, Machine Learning, and..."
1,Model-Based Clustering and Classification for ...
2,Spatial Modeling in GIS and R for Earth and En...
3,Statistical Machine Learning: A Unified Framew...
4,Data Science for Transport: A Self-Study Guide...
...,...
267,Python Machine Learning: Machine Learning and ...
268,Machine Learning for Business: Using Amazon Sa...
269,TensorFlow Machine Learning Cookbook: Over 60 ...
270,MATLAB Machine Learning Recipes: A Problem-Sol...


In [158]:
#Xtrain, Xval, ytrain, yval = train_test_split(treinar, y, test_size=0.50, random_state=42)
Xtrain, Xval = treinar[:80], treinar[80:]
ytrain, yval = y[:80], y[80:]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((80, 2), (192, 2), (80,), (192,))

In [19]:
#texto para números
from sklearn.feature_extraction.text import TfidfVectorizer

In [179]:
title_train = df_limpo[:80]['title']
title_val = df_limpo[80:]['title']

title_vec = TfidfVectorizer(min_df = 2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

In [180]:
title_bow_train.shape, title_bow_val.shape

((80, 100), (192, 100))

In [161]:
title_bow_train

<80x100 sparse matrix of type '<class 'numpy.float64'>'
	with 664 stored elements in Compressed Sparse Row format>

In [117]:
#porcentagem da matriz que é composta por 0
1 - 658/(80*120)

0.9314583333333333

In [35]:
from scipy.sparse import hstack, vstack

In [181]:
#juntando os o treino com o tf-idf
Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [182]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((80, 102), (192, 102))

In [183]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight='balanced', n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=6,
                       random_state=0)

In [184]:
p = mdl.predict_proba(Xval_wtitle)[:,1]

In [166]:
p

array([0.615     , 0.61426741, 0.548     , 0.422     , 0.55851799,
       0.503     , 0.39477863, 0.64525895, 0.509     , 0.33876712,
       0.39918091, 0.566     , 0.524     , 0.727     , 0.53311374,
       0.704     , 0.55890774, 0.427     , 0.571     , 0.57      ,
       0.32435902, 0.499275  , 0.57      , 0.512     , 0.43439772,
       0.544     , 0.55972512, 0.26      , 0.6913854 , 0.6876375 ,
       0.576     , 0.33      , 0.43659332, 0.41361929, 0.36484828,
       0.4856375 , 0.618     , 0.551     , 0.6206375 , 0.443     ,
       0.3708147 , 0.541     , 0.528     , 0.515     , 0.44905363,
       0.6266375 , 0.64445358, 0.515     , 0.3656375 , 0.665     ,
       0.643     , 0.64977863, 0.3666985 , 0.378     , 0.599     ,
       0.44514787, 0.60863883, 0.323     , 0.505     , 0.51545513,
       0.59647816, 0.388     , 0.81      , 0.42382443, 0.267     ,
       0.3586375 , 0.602     , 0.540275  , 0.33050374, 0.634     ,
       0.48430855, 0.601     , 0.387     , 0.3708147 , 0.41   

In [178]:
ytrain.value_counts()

0    51
1    29
Name: target, dtype: int64

In [80]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [185]:
average_precision_score(yval,p)

0.17585895310304805

In [186]:
roc_auc_score(yval, p)

0.48799313893653506

## Active Learning

In [197]:
df_unlabeled = pd.read_excel('sem_duplicate.xlsx')
df_unlabeled = df_unlabeled[df_unlabeled['target'] == 0].dropna(how = 'all')
df_unlabeled.drop(columns = ['target'], axis = 1, inplace = True)

In [193]:
mask = ['classification','reviews']
features_u = df_unlabeled[mask]

In [190]:
df_limpo_u = pd.DataFrame(index = df_unlabeled.index)
df_limpo_u['title'] = df_unlabeled['title']

In [191]:
title_u = df_limpo_u['title']
title_bow_u = title_vec.transform(title_u)

In [192]:
title_bow_u

<210x100 sparse matrix of type '<class 'numpy.float64'>'
	with 1587 stored elements in Compressed Sparse Row format>

In [194]:
Xu_title = hstack([features_u, title_bow_u])

In [195]:
Xu_title

<210x102 sparse matrix of type '<class 'numpy.float64'>'
	with 2006 stored elements in COOrdinate format>

In [196]:
pu = mdl.predict_proba(Xu_title)[:,1]

In [198]:
df_unlabeled['p'] = pu

In [199]:
df_unlabeled

,title,image,classification,url,reviews,p
4,Data Science for Transport: A Self-Study Guide...,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Data-Science-Transport-...,1,0.110637
5,Go Machine Learning Projects: Eight projects d...,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Go-Machine-Learning-Pro...,1,0.099674
6,Java Data Science Cookbook,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Java-Science-Cookbook-R...,1,0.140468
7,Leading a Data Driven Organization: A Practica...,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Leading-Data-Driven-Org...,1,0.092411
8,Learn Data Mining Through Excel: A Step-by-Ste...,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Learn-Mining-Through-Ex...,1,0.070306
...,...,...,...,...,...,...
267,Python Machine Learning: Machine Learning and ...,https://images-na.ssl-images-amazon.com/images...,3.1,https://www.amazon.com/Python-Machine-Learning...,12,0.341231
268,Machine Learning for Business: Using Amazon Sa...,https://images-na.ssl-images-amazon.com/images...,3.0,https://www.amazon.com/Machine-Learning-Busine...,1,0.527000
269,TensorFlow Machine Learning Cookbook: Over 60 ...,https://images-na.ssl-images-amazon.com/images...,3.0,https://www.amazon.com/TensorFlow-Machine-Lear...,2,0.237468
270,MATLAB Machine Learning Recipes: A Problem-Sol...,https://images-na.ssl-images-amazon.com/images...,2.9,https://www.amazon.com/MATLAB-Machine-Learning...,5,0.417000


In [205]:
mask_u = (df_unlabeled['p'] >= 0.40)&(df_unlabeled['p'] <= 0.60)
mask_u.sum()

93

In [203]:
df_unlabeled[mask_u]

,title,image,classification,url,reviews,p
29,Machine Learning and Data Science: An Introduc...,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Machine-Learning-Data-S...,4,0.506516
82,The Ultimate Data and AI Guide: 150 FAQs About...,https://images-na.ssl-images-amazon.com/images...,4.6,https://www.amazon.com/Ultimate-Data-Guide-Art...,26,0.548000
88,"Practical Deep Learning for Cloud, Mobile, and...",https://images-na.ssl-images-amazon.com/images...,4.6,https://www.amazon.com/Practical-Learning-Clou...,59,0.509000
92,Python for Finance: Mastering Data-Driven Finance,https://images-na.ssl-images-amazon.com/images...,4.6,https://www.amazon.com/Python-Finance-Masterin...,95,0.524000
94,Introduction to Computation and Programming Us...,https://images-na.ssl-images-amazon.com/images...,4.6,https://www.amazon.com/Introduction-Computatio...,250,0.533114
101,Evaluating Learning Algorithms: A Classificati...,https://images-na.ssl-images-amazon.com/images...,4.5,https://www.amazon.com/Evaluating-Learning-Alg...,2,0.499275
103,Hands-On Artificial Intelligence for Cybersecu...,https://images-na.ssl-images-amazon.com/images...,4.5,https://www.amazon.com/Hands-Artificial-Intell...,5,0.512000
105,Information Visualization: Perception for Desi...,https://images-na.ssl-images-amazon.com/images...,4.5,https://www.amazon.com/Information-Visualizati...,8,0.544000
115,Fundamentals of Machine Learning for Predictiv...,https://images-na.ssl-images-amazon.com/images...,4.5,https://www.amazon.com/Fundamentals-Machine-Le...,71,0.485637
121,Python Data Science Handbook: Essential Tools ...,https://images-na.ssl-images-amazon.com/images...,4.5,https://www.amazon.com/Python-Data-Science-Han...,253,0.541000


In [206]:
dificeis = df_unlabeled[mask_u]

In [207]:
aleatorios = df_unlabeled[~mask_u].sample(31)

In [210]:
pd.concat([dificeis, aleatorios]).to_excel('active_label1.xlsx')

In [209]:
dificeis

,title,image,classification,url,reviews,p
29,Machine Learning and Data Science: An Introduc...,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Machine-Learning-Data-S...,4,0.506516
82,The Ultimate Data and AI Guide: 150 FAQs About...,https://images-na.ssl-images-amazon.com/images...,4.6,https://www.amazon.com/Ultimate-Data-Guide-Art...,26,0.548000
83,Machine Learning: Jump on the Way to the Futur...,https://m.media-amazon.com/images/I/41wfVYeu8b...,4.6,https://www.amazon.com/Machine-Learning-Artifi...,27,0.422000
88,"Practical Deep Learning for Cloud, Mobile, and...",https://images-na.ssl-images-amazon.com/images...,4.6,https://www.amazon.com/Practical-Learning-Clou...,59,0.509000
91,Machine Learning with R: Expert techniques for...,https://images-na.ssl-images-amazon.com/images...,4.6,https://www.amazon.com/Machine-Learning-techni...,94,0.566000
...,...,...,...,...,...,...
265,Electrical Costs with Rsmeans Data: 60030,https://images-na.ssl-images-amazon.com/images...,3.3,https://www.amazon.com/Electrical-Costs-RSMean...,6,0.465832
266,Machine Learning for Finance: Principles and p...,https://images-na.ssl-images-amazon.com/images...,3.3,https://www.amazon.com/Machine-Learning-Financ...,9,0.434468
268,Machine Learning for Business: Using Amazon Sa...,https://images-na.ssl-images-amazon.com/images...,3.0,https://www.amazon.com/Machine-Learning-Busine...,1,0.527000
270,MATLAB Machine Learning Recipes: A Problem-Sol...,https://images-na.ssl-images-amazon.com/images...,2.9,https://www.amazon.com/MATLAB-Machine-Learning...,5,0.417000
